In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/kaggle/input/adult-income-dataset/adult.csv")

In [ ]:
df

In [ ]:
df.education.value_counts()

In [ ]:
df.occupation.value_counts()

In [ ]:
df = pd.concat([df.drop("occupation", axis=1), pd.get_dummies(df['occupation'], dtype=int).add_prefix("occupation_")], axis=1)
df = pd.concat([df.drop("workclass", axis=1), pd.get_dummies(df['workclass'], dtype=int).add_prefix("workclass_")], axis=1)
df = df.drop("education",axis=1)
df = pd.concat([df.drop("marital-status", axis=1), pd.get_dummies(df['marital-status'], dtype=int).add_prefix("marital-status_")], axis=1)
df = pd.concat([df.drop("relationship", axis=1), pd.get_dummies(df['relationship'], dtype=int).add_prefix("relationship_")], axis=1)
df = pd.concat([df.drop("race", axis=1), pd.get_dummies(df['race'], dtype=int).add_prefix("race_")], axis=1)
df = pd.concat([df.drop("native-country", axis=1), pd.get_dummies(df['native-country'], dtype=int).add_prefix("native-country_")], axis=1)

In [ ]:
df

In [ ]:
df['gender'] = df['gender'].apply(lambda x : 1 if x == 'Male' else 0)
df['income'] = df['income'].apply(lambda x : 1 if x == '>50K' else 0)

In [ ]:
df.columns.values

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(18,12))
sns.heatmap(df.corr(), annot= False , cmap='coolwarm')

In [ ]:
df.corr()

In [ ]:
correlations = df.corr()['income'].abs()
sorted_correlations = correlations.sort_values()
num_cols_to_drop = int(0.8*len(df.columns))
cols_to_drop = sorted_correlations.iloc[:num_cols_to_drop].index
df_dropped = df.drop(cols_to_drop,axis=1)

In [ ]:
df_dropped

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(15,10))
sns.heatmap(df_dropped.corr(), annot= True , cmap='coolwarm')

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import train_test_split 

df = df.drop('fnlwgt',axis=1)

train_df,test_df = train_test_split(df,test_size=0.2)

In [ ]:
train_df

In [ ]:
test_df

In [ ]:
train_x = train_df.drop("income",axis=1)
train_y = train_df['income']

test_x = test_df.drop("income",axis=1)
test_y = test_df['income']

In [ ]:
forest = RandomForestClassifier()

forest.fit(train_x,train_y)


In [ ]:
forest.score(test_x,test_y)

In [ ]:
forest.feature_importances_

In [ ]:
forest.feature_names_in_

In [ ]:
importance = dict(zip(forest.feature_names_in_,forest.feature_importances_))
importance = {k: v for k , v in sorted (importance.items(), key=lambda x: x[1], reverse = True)}

In [ ]:
importance

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "n_estimators": [50, 100, 250],
    "max_depth": [5, 20, 30, None],
    "min_samples_split": [2, 4],
    "max_features": ["sqrt", "log2"]
}

grid_search = GridSearchCV(estimator=RandomForestClassifier(),
                           param_grid=param_grid, verbose=10)

In [ ]:
grid_search.fit(train_x,train_y)

In [ ]:
grid_search.best_estimator_

In [ ]:
forest = grid_search.best_estimator_

In [ ]:
forest.score(test_x,test_y)

In [ ]:
importance = dict(zip(forest.feature_names_in_,forest.feature_importances_))
importance = {k: v for k , v in sorted (importance.items(), key=lambda x: x[1], reverse = True)}

In [ ]:
importance